In [1]:
from data import Wiki_Dataset
from model import glove_model
import torch
import numpy as np
import torch.optim as optim
from tqdm import tqdm

In [2]:
# 导入模型参数
import config as argumentparser
config = argumentparser.ArgumentParser()

In [3]:
# 设置GPU
if config.cuda and torch.cuda.is_available():
    torch.cuda.set_device(config.gpu)

In [4]:
torch.cuda.is_available()

False

In [5]:
# 导入训练集
wiki_dataset = Wiki_Dataset(min_count=config.min_count,window_size=config.window_size)
training_iter = torch.utils.data.DataLoader(dataset=wiki_dataset,
                                            batch_size=config.batch_size,
                                            shuffle=True,
                                            num_workers=2)

In [6]:
model = glove_model(len(wiki_dataset.word2id),config.embed_size,config.x_max,config.alpha)

In [7]:
# 将模型送进gpu
if config.cuda and torch.cuda.is_available():
    torch.cuda.set_device(config.gpu)
    model.cuda()

In [8]:
# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [9]:
loss= -1
for epoch in range(config.epoch):
    process_bar = tqdm(training_iter) # tqdm训练
    for data, label in process_bar:
        w_data = torch.Tensor(np.array([sample[0] for sample in data])).long()
        v_data = torch.Tensor(np.array([sample[1] for sample in data])).long()
        if config.cuda and torch.cuda.is_available():
            w_data = w_data.cuda()
            v_data = v_data.cuda()
            label = label.cuda()
        loss_now =model(w_data,v_data,label)
        if loss==-1:
            loss = loss_now.data.item()
        else:
            loss = 0.95*loss+0.05*loss_now.data.item() # 平滑loss
        process_bar.set_postfix(loss=loss) # 输出loss
        process_bar.update()
        optimizer.zero_grad() # 梯度更新
        loss_now.backward()
        optimizer.step()
model.save_embedding()

100%|██████████| 287217/287217 [1:48:43<00:00, 44.03it/s, loss=1.28]  


TypeError: glove_model.save_embedding() missing 2 required positional arguments: 'word2id' and 'file_name'